In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import numpy as np
from sklearn import decomposition
import matplotlib.pyplot as plt
import re
import nltk
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from trankit import Pipeline
import json

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
p = Pipeline('hebrew')

with open("./../Poems_And_Outputs/Poems_Unparsed.JSON", "r", encoding='utf-8') as file:
    data = json.load(file)
    
poems = [poem["content"] for poem in data]
names = [poem["poem"] for poem in data]

הכנת אוסף מילות מפתח מכל שיר (בלי "וגם" וכאלה)

In [1]:
poems_posed = []
for i,poem in enumerate(poems):
    if i<100:
      poems_posed.append(p(poem))
    if i==0:
        print(poems_posed[0])

with open("./../Poems_And_Outputs/all_songs_processed.JSON", 'w', encoding='utf-8') as f:
    json.dump(poems_posed, f, indent=4, ensure_ascii=False)

NameError: name 'poems' is not defined

In [129]:
def tokenize():
    with open("./../Poems_And_Outputs/all_songs_processed.JSON", 'r', encoding='utf-8') as f:
        songs_parsed = json.load(f)
        res = []
        for song in songs_parsed:
            res.append(_tokenize(song))
        return res

def _tokenize(song_dict):
    tokens = []
    if isinstance(song_dict, dict):
        for key, value in song_dict.items():
            if key == 'tokens':
                tokens.extend(get_tokens(song_dict["tokens"]))
            elif isinstance(value, dict):
                tokens.extend(_tokenize(value))
            elif isinstance(value, list):
                for item in value:
                    if isinstance(item, dict):
                        tokens.extend(_tokenize(item))
    return tokens

def get_tokens(tokens):
    stop_words_types = ["PUNCT", "PRON", "ADP","DET", "CCONJ", "SCONJ", "PART", "SYM", "X", "NUM"]
    included_types = ["NOUN", "VERB"]
    tokens_from_list = []
    for token in tokens:
        if "upos" not in token and "expanded" in token:
            token["upos"] = token["expanded"][0]["upos"]
            token["lemma"] = token["expanded"][0]["lemma"]
        if "upos" in token and token["upos"] in included_types:
            tokens_from_list.append(token["lemma"])
    return tokens_from_list


אימון המודל על מילות מפתח מכל אוספי השירים

In [131]:
poems_processed = tokenize()
poems_processed = [" ".join(poem) for poem in poems_processed]
x_train, x_hold = train_test_split(poems_processed, test_size=0.3, random_state=111)
vectorizer_tf = TfidfVectorizer(max_df=0.7, min_df=2,max_features=10000, use_idf=False, norm=None)
vectorizer_tf.lowercase = False
tf_vectors = vectorizer_tf.fit_transform(x_train)

זה הריבויים של כל מילת מפתח שנבחרה בכל שיר:

In [132]:
tf_vectors.A

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 2., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

וזה המילים שייצאו:

In [133]:
vectorizer_tf.get_feature_names_out()

array(['__', 'אב', 'אבא', 'אבד', 'אביב', 'אבך', 'אבן', 'אבר', 'אגדה',
       'אדם', 'אדמה', 'אדן', 'אדע', 'אדרה', 'אהב', 'אהבה', 'אהה', 'אהל',
       'און', 'אוצר', 'אור', 'אורה', 'אורך', 'אורם', 'אות', 'אזוב', 'אזן',
       'אח', 'אחז', 'אחיך', 'אחים', 'אחרית', 'איבד', 'אין', 'איש', 'אישר',
       'אכל', 'אל', 'אלהים', 'אלון', 'אלם', 'אם', 'אמונה', 'אמן', 'אמר',
       'אמת', 'אנחה', 'אספה', 'אף', 'אפון', 'אפיק', 'אפך', 'אפס', 'אצא',
       'אצבע', 'ארא', 'ארב', 'ארג', 'ארון', 'ארך', 'ארץ', 'אש', 'אשא',
       'אשנב', 'אשר', 'בא', 'בדק', 'ביטח', 'ביקר', 'ביקש', 'בית', 'בכה',
       'בכי', 'בל', 'בלב', 'בן', 'בסתר', 'בעב', 'בער', 'בקר', 'ברזל',
       'ברך', 'ברכה', 'בשר', 'בת', 'גאלה', 'גב', 'גבול', 'גבעת', 'גבר',
       'גג', 'גדל', 'גדר', 'גוון', 'גויל', 'גוים', 'גוף', 'גופה', 'גחל',
       'גחלה', 'גיל', 'גל', 'גלגל', 'גלה', 'גלות', 'גמל', 'גן', 'גנב',
       'גנוז', 'גע', 'געגוע', 'גף', 'גפה', 'דבר', 'דום', 'דומייה', 'דור',
       'דיבר', 'דלות', 'דם', 'דמדום', 'דמות', 'דמם', 'דממ

אימון מודל שמקשר בין מילות מפתח בשירים השונים ויוצר נושאים על בסיס מילים משותפות:

In [134]:
lda = decomposition.LatentDirichletAllocation(n_components=6, max_iter=5, learning_method='batch', n_jobs=-1, random_state=111).fit(tf_vectors)
W1 = lda.fit_transform(tf_vectors)
H1 = lda.components_

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

זה ההסתברות של כל שיר להיות בנושא מסויים מבין 6 הנושאים:

In [135]:
W1

array([[9.82563461e-03, 9.50782989e-01, 9.85082080e-03, 9.82368588e-03,
        9.84558205e-03, 9.87128763e-03],
       [1.04763938e-03, 9.94754661e-01, 1.04900029e-03, 1.04683214e-03,
        1.05059765e-03, 1.05126969e-03],
       [2.74391343e-03, 2.74772422e-03, 9.86261027e-01, 2.74570950e-03,
        2.75080184e-03, 2.75082367e-03],
       [3.09295421e-03, 3.10727180e-03, 3.10948737e-03, 3.10001490e-03,
        3.10672367e-03, 9.84483548e-01],
       [6.70207864e-03, 6.70031496e-03, 6.70643093e-03, 9.66487849e-01,
        6.70398171e-03, 6.69934462e-03],
       [5.22929242e-03, 5.22264721e-03, 5.23992969e-03, 9.73817587e-01,
        5.24566683e-03, 5.24487720e-03],
       [2.69963390e-03, 2.70580268e-03, 2.70603369e-03, 2.70648365e-03,
        9.86475776e-01, 2.70626988e-03],
       [3.09501449e-03, 3.10132821e-03, 3.10251130e-03, 9.84502349e-01,
        3.09903904e-03, 3.09975749e-03],
       [3.41662867e-03, 3.42238911e-03, 3.43100356e-03, 3.42264053e-03,
        9.82881363e-01, 

וזה 6 הנושאים:

In [136]:
num_words = 10
vocab = np.array(vectorizer_tf.get_feature_names_out())
top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]
topic_words = ([top_words(t) for t in H1])
topics = [' '.join(t) for t in topic_words]

In [137]:
topics

['עולם אין יום אלהים רוח פה עב נפש אם אמר',
 'אין נפש רוח עין קול חיים יום אלהים אבד עם',
 'אור נפש עין לב שמש יד יום בן ראש חיים',
 'אור נפל בא קול קרן נפש יד הלך לב ילד',
 'חיים לב עין פנים ראה כוכב אור בן היה נפש',
 'איש עין חיים רוח דמעה ידע עולם קול דם נפש']